In [ ]:
# 1/v = 1/s * B *(Km/Vmax) + A/Vmax

In [2]:
import pandas as pd
data = dict(s = [0.033,0.05,0.066,0.1,0.2],
            v_norm = [0.01008,0.01215,0.01341,0.01521,0.01728],
            v_inib= [0.00657,0.00837,0.00981,0.01179,0.01476]
           )
df = pd.DataFrame(data)
df

,s,v_norm,v_inib
0,0.033,0.01008,0.00657
1,0.050,0.01215,0.00837
2,0.066,0.01341,0.00981
3,0.100,0.01521,0.01179
4,0.200,0.01728,0.01476


In [3]:
from bokeh.plotting import figure, show, output_notebook
param = dict(width = 700, height = 250,
             tools = 'wheel_zoom, reset,pan, box_zoom',
             tooltips = [( 'substrate',   '@s'), ( 'velocity',   '@v')],
            )
output_notebook()

Loading BokehJS ...

In [6]:
p = figure(x_axis_label='Concentrazione substrato',
           y_axis_label='Velocità di reazione',
           **param,
           )
p.circle( 's', 'v_norm', source=df )
p.square('s', 'v_inib', source=df) # Plotto i punti (s,v)
show( p )

In [8]:
df['1/s'] = 1 / df['s']
df['1/v_norm'] = 1 / df['v_norm']
df['1/v_inib']= 1/df['v_inib']
df

,s,v_norm,v_inib,1/s,1/v_norm,1/v_inib
0,0.033,0.01008,0.00657,30.303030,99.206349,152.207002
1,0.050,0.01215,0.00837,20.000000,82.304527,119.474313
2,0.066,0.01341,0.00981,15.151515,74.571216,101.936799
3,0.100,0.01521,0.01179,10.000000,65.746220,84.817642
4,0.200,0.01728,0.01476,5.000000,57.870370,67.750678


In [15]:
from scipy.stats import linregress
slope_norm, intercept_norm, r_value, p_value, std_err = linregress( df['1/s'], df['1/v_norm'] )
slope, intercept   

(1.6373268490967405, 49.59365880485513)

In [25]:
from scipy.stats import linregress
slope_inib, intercept_inib, r_value, p_value, std_err = linregress( df['1/s'], df['1/v_inib'] )
'The slope of the inibited enzyme is {}, and the intercept of the inibited enzyme is {}'.format(slope_inib, intercept_inib)  

'The slope of the inibited enzyme is 3.3480784248271647, and the intercept of the inibited enzyme is 51.36366109799226'

In [13]:
from numpy import array 

In [23]:
#faccio un grafico con le 2 rette per capire il tipo di inibizione
p = figure(title = 'Linearizzazione di Lineweaver-Burk',
            x_axis_label= '1/s', 
            y_axis_label= '1/v',
            **param,
            )
p.circle( '1/s', '1/v_norm', source=df )
xmax = df['1/s'].max() 
x = array( [-3, xmax] )       
y = x * slope_norm + intercept_norm 
p.line( x, y, color='green')

p.square( '1/s', '1/v_inib', source=df)
     
y = x * slope_inib + intercept_inib 
p1.line( x, y, color='red')
show( p1 )